#Code implementation of the paper "To Act or React?"
https://arxiv.org/pdf/1906.11932v1.pdf

Connect: ahmedrashidqazi@gmail.com

###Libraries

In [4]:
import os
import sys
import pandas as pd
import numpy as np
import sklearn
from google.cloud import bigquery
import re

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [5]:
client = bigquery.Client('myproject123-319010')

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Data
##Time Frame - 01/2015 - 10/2018

###DA = Most Active 3 thousand Subreddits based on the average number of posts per month

In [7]:
DA = []
DA = pd.read_csv("/content/gdrive/MyDrive/Most Active 3000 Subreddit Names.csv")
DA

,subreddit,a
0,AskReddit,210116.4
1,AutoNewspaper,183784.6
2,RocketLeagueExchange,146558.1
3,CircleofTrust,142087.7
4,StocksInFocus,128409.5
...,...,...
2995,Rainmeter,674.0
2996,funwithwords,673.4
2997,Ooer,673.1
2998,u_b206050de3,673.0


###DB = 37 Quarantined/Banned Subreddits

In [103]:
DB = ['Braincels',
 'TheRedPill',
 'Ice_Poseidon',
 'CringeAnarchy',
 'TimeToGo',
 'Ice_Poseidon2',
 'DebateAltRight',
 'OffensiveMemes',
 'thinspocommunity',
 'ZOG',
 'EDFood',
 'spacedicks',
 'whitebeauty',
 'Clown__World',
 'Clownworldwar',
 'FULLCOMMUNISM',
 'GentilesUnited',
 'NorthwestFront',
 'okbuddyanarchy',
 'fragilejewishredditor',
 'Identitarians',
 'niggas',
 'holocaust',
 'tha_pit_pit',
 'Mayo_town',
 'CringeChaos',
 'kangznsheeit',
 'cursedx100images',
 'SubforWhitePeopleOnly',
 'cursedx3images',
 'The_Donald',
 'ChapoTrapHouse',
 'AmericanJewishPower',
 'honkler',
 'WatchPeopleDie',
 'Physical_Removal',
 'altright'] 

####The code below was used to extract the above 36 subreddits

In [17]:
# DB = []

# query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND quarantine = true """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   x = x['subreddit']
#   print(x.unique())
#   for k in x.unique():
#     if k not in DB:
#       DB.append(k)

# query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.2015_12  WHERE quarantine = true """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   x = x['subreddit']
#   print(x.unique())
#   for k in x.unique():
#     if k not in DB:
#       DB.append(k)

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# years = ['2018_08', '2019_01']
# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.""" + year + """ WHERE quarantine = true """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       x = x['subreddit']
#       print(x.unique())
#       for k in x.unique():
#         if k not in DB:
#           DB.append(k)
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break


# DB.append("WatchPeopleDie")
# DB.append("Physical_Removal")

 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_07 WHERE quarantine = true 
The query for the month 2018_07
Empty Dataframe



 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_08 WHERE quarantine = true 
The query for the month 2018_08
Empty Dataframe



 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_09 WHERE quarantine = true 
The query for the month 2018_09
['Braincels' 'TheRedPill' 'Ice_Poseidon' 'CringeAnarchy']



 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_10 WHERE quarantine = true 
The query for the month 2018_10
['TheRedPill' 'CringeAnarchy' 'Ice_Poseidon' 'Braincels']



 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_11 WHERE quarantine = true 
The query for the month 2018_11
['Braincels' 'TheRedPill' 'Ice_Poseidon' 'CringeAnarchy']



 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_12 WHERE quarantine = true 
The query for the month 2018_12
['Braincels' 'TheRedPill' 'Ice_Poseidon' 'CringeAnarchy']



 SELECT subreddit FROM fh-bigquery.reddi

In [18]:
print("Total number of banned/quarantined subreddits", len(DB))
DB

Total number of banned/quarantined subreddits 36


['Braincels',
 'TheRedPill',
 'Ice_Poseidon',
 'CringeAnarchy',
 'TimeToGo',
 'Ice_Poseidon2',
 'DebateAltRight',
 'OffensiveMemes',
 'thinspocommunity',
 'ZOG',
 'EDFood',
 'spacedicks',
 'whitebeauty',
 'Clown__World',
 'Clownworldwar',
 'FULLCOMMUNISM',
 'GentilesUnited',
 'NorthwestFront',
 'okbuddyanarchy',
 'fragilejewishredditor',
 'Identitarians',
 'niggas',
 'holocaust',
 'tha_pit_pit',
 'Mayo_town',
 'CringeChaos',
 'kangznsheeit',
 'cursedx100images',
 'SubforWhitePeopleOnly',
 'cursedx3images',
 'The_Donald',
 'ChapoTrapHouse',
 'AmericanJewishPower',
 'honkler',
 'WatchPeopleDie',
 'Physical_Removal']

###DH = Hateful Subreddits

####Extracting the text, title, score and number of comments from the subreddit r/AgainstHateSubreddits

In [38]:
# text = pd.DataFrame([], columns = ['selftext', 'title'])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND (subreddit = 'AgainstHateSubreddits') """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.2015_12  WHERE subreddit = 'AgainstHateSubreddits' """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.""" + year + """ WHERE subreddit = 'AgainstHateSubreddits' """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       print(x)
#       text = pd.concat([text,x])
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break

# text.to_csv("/content/gdrive/MyDrive/AgainstHateSubredditsText.csv", index = False)

 SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.2016_01 WHERE subreddit = 'AgainstHateSubreddits' 
The query for the month 2016_01
                                              selftext  ... num_comments
0                                                       ...            1
1                                                       ...            4
2                                                       ...            3
3                                            [removed]  ...            1
4                                                       ...            9
..                                                 ...  ...          ...
203  http://np.reddit.com/r/GenderCritical/comments...  ...           22
204                                                     ...            0
205                                                     ...            1
206                                                     ...           17
207                                    

####Extracting the text, title, score and number of comments from the subreddit r/SubredditDrama

In [80]:
# text = pd.DataFrame([], columns = ['selftext', 'title'])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND (subreddit = 'SubredditDrama') """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.2015_12  WHERE subreddit = 'SubredditDrama' """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.""" + year + """ WHERE subreddit = 'SubredditDrama' """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       print(x)
#       text = pd.concat([text,x])
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break

# text.to_csv("/content/gdrive/MyDrive/SubredditDramaText.csv", index = False)

 SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.2016_01 WHERE subreddit = 'SubredditDrama' 
The query for the month 2016_01
                                               selftext  ... num_comments
0     In the interest of getting people to actually ...  ...           97
1     [Full thread](https://np.reddit.com/r/gaming/c...  ...            5
2     The drama starts when long-time contributor [C...  ...          237
3     Open carry of firearms is already a loaded top...  ...          261
4                                                        ...            2
...                                                 ...  ...          ...
1473                                                     ...           14
1474                                                     ...          145
1475                                          [deleted]  ...          437
1476                                          [deleted]  ...           21
1477                               

####Analysing the subreddit to extract names of hateful subreddits from AgainstHateSubreddits

In [104]:
AHS = pd.read_csv("/content/gdrive/MyDrive/AgainstHateSubredditsText.csv")
AHS

,selftext,title,score,num_comments
0,NaN,/r/WorldPolitics now linking ONLY to anti-immi...,74.0,30.0
1,NaN,A hate subreddit brigade.,45.0,16.0
2,NaN,Action Taken by mods - Claims that /r/Europe i...,30.0,7.0
3,NaN,"""We need to start a trend on twatter #RemoveKe...",10.0,3.0
4,NaN,Test post please ignore,1.0,1.0
...,...,...,...,...
22973,[removed],What the actual f*ck is this sub,1.0,0.0
22974,[removed],Some thoughts I've got from being on this sub,1.0,0.0
22975,[removed],META: Should direct reddit links be banned on ...,18.0,8.0
22976,[removed],Pro-China Protestors Nazi salute in Vancouver ...,1.0,0.0


In [106]:
#Posts below a score of 100 will be dropped
AHS = AHS.sort_values(by = "score", ascending = False)
AHSNew = AHS[(AHS['score'] > 50) & (AHS['num_comments'] > 25)]
AHSNew['selftext'] = AHSNew['selftext'].fillna(" ")
AHSNew

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,selftext,title,score,num_comments
7380,,The ACTUAL FUCKING NAZIS AND WHITE NATIONALIST...,43178.0,2141.0
14743,[deleted],"The_Donald is a violent, extremist group being...",38130.0,2071.0
6231,,HanAssholeSolo wished for people to be doxxed ...,30570.0,1710.0
10093,"For those who may not be aware, /r/The_donald,...",Reddit admins have confirmed they are comforta...,30338.0,2017.0
5426,,/r/The_Donald Has Built A Document With The Ad...,28866.0,2375.0
...,...,...,...,...
19366,,/r/frenworld citing the 14 words,51.0,32.0
1137,Proud of it even!\n\nhttps://www.reddit.com/r/...,Trump supporter admits to /r/the_donald is a h...,51.0,49.0
2301,,"On transgender people: ""You can't change your ...",51.0,35.0
359,,/r/TumblrInAction has removed some mods to hel...,51.0,46.0


####Analysing the subreddit to extract names of hateful subreddits from SubredditDrama

In [107]:
SD = pd.read_csv("/content/gdrive/MyDrive/SubredditDramaText.csv")
SD

,selftext,title,score,num_comments
0,NaN,Should someone kill themselves if they have a ...,37.0,97.0
1,NaN,2XC debates with OP as to whether a women only...,158.0,447.0
2,NaN,/r/occult is arguing about something. I have n...,1274.0,473.0
3,NaN,Refugee Drama in r/Islam,10.0,56.0
4,NaN,/u/bestofoutrageculture stickies a post to the...,248.0,209.0
...,...,...,...,...
70584,[deleted],r/AskTeenGirls mod having a fit over guys answ...,3.0,8.0
70585,[removed],r/Dota2 Mods turn off comments after Chinese c...,1.0,4.0
70586,[removed],FYI r/atheism is handing out lifetime bans for...,1.0,5.0
70587,[removed],R/History Perma Banned for asking about Moors,0.0,28.0


In [108]:
#Posts below a score of 100 will be dropped
SD = SD.sort_values(by = "score", ascending = False)
SDNew = SD[(SD['score'] > 200) & (SD['num_comments'] > 100)]
SDNew['selftext'] = SDNew['selftext'].fillna(" ")
SDNew

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,selftext,title,score,num_comments
68129,/r/The_Donald has been quarantined. Discuss th...,/r/The_Donald has been quarantined. Discuss th...,65481.0,24078.0
60526,,r/fuckthealtright mod made a detailed post of ...,51349.0,3311.0
53887,"It's literally the ENTIRE comment section, but...","r/The_Donald is imploding, following Trump's p...",41074.0,6525.0
44455,The_Donald has [gone private](/r/the_donald). ...,The_donald has gone private in protest of thei...,39328.0,10391.0
41072,/r/altright has been banned by the admins for ...,/r/altright has been banned. Discuss this here!,34168.0,8274.0
...,...,...,...,...
31386,1. [Joke comparing the protester to a difficul...,"""Trigglypuff"" video posted to /r/YoutubeHaiku....",201.0,296.0
5585,,"""Take your preferred gender pronouns and stick...",201.0,558.0
35905,,Bad mathematics in /r/badmathematics. Ignoranc...,201.0,273.0
4465,,[Dramatic Happenings] /r/LeagueofLegends mods ...,201.0,247.0


####Function to extract reddit names from text

In [109]:
def clean_string(st):
  st = st.lower()
  st = st.replace(",","")
  st = st.replace(".","")
  st = st.replace(":","")
  st = st.replace("!","")
  st = st.replace("'","")
  st = st.replace(")","")
  st = st.replace("(","")
  return st


def find_matches(text):
  matches = {}
  for i in range(0, len(text)):
    m = []
    x = text.iloc[i]

    y = re.findall(r"/r/([^\s/]+)", x['title'])
    if len(y) > 0:
      y = clean_string(y[0])
      m.append(y)

    z = re.findall(r"/r/([^\s/]+)", x['selftext'])
    if len(z) > 0:
      z = clean_string(z[0])
      m.append(z)

    for i in m:
      if i not in matches.keys():
        matches[i] = 1
      else:
        matches[i] = matches[i] + 1
      break
  
  print("Matches found:", len(matches))
  print(matches)
  print("\n\n")

  temp_db = []
  for i in DB:
    temp_db.append(i.lower())

  matcheswithdb = []
  for i in matches.keys():
    if i in temp_db:
      matcheswithdb.append(i)

  print("Matches also found in quarantined/banned subreddits:", len(matcheswithdb))
  print(len(matcheswithdb), matcheswithdb)
  print("\n\n")

  print("Removing matches found in DB")
  for i in matcheswithdb:
    matches.pop(i)

  print("Final matches:", len(matches))
  print(matches)
  return matches

In [110]:
AHS_Final = find_matches(AHSNew)

Matches found: 256
{'the_donald': 190, 'physical_removal': 15, 'altright': 32, 'incels': 18, 'imgoingtohellforthis': 8, 'uncensorednews': 33, 'onguardforthee': 2, 'whiterights': 17, 'waterniggas': 1, 'blog': 1, 'greatawakening': 3, 'milliondollarextreme': 28, 'all': 4, 'conservative': 9, 'subredditdrama': 1, 'honkler': 1, 'race_realism': 1, 'obese_niggers': 1, 'topmindsofreddit': 1, 'shitpoliticssays': 2, 'metacanada': 18, 'conspiracy': 45, 'kiachatroom': 1, 'landofbuddies': 1, 'offensivespeech': 2, 'chadfish': 2, 'politics': 3, 'cringanarchy': 1, 'games': 1, 'canada': 7, 'stonetoss': 2, 'majorityrights': 1, 'drama': 6, 'antifa': 2, 'sjwhate': 4, 'clownworldwar2': 1, 'askthedonald': 1, 'racialrealism': 1, 'hillaryforprison': 1, 'catholicism': 5, 'israel': 9, 'rightwinglgbt': 2, 'clownworldwar': 5, 'blackcrime': 2, 'cringeanarchy': 33, 'modnews': 1, 'justiceserved': 1, 'transgenderkids': 1, 'the_donaldunleashed': 1, 'debatefascism': 2, 'againsthatesubreddits': 8, 'incest': 1, 'israelpal

In [111]:
SD_Final = find_matches(SDNew)

Matches found: 1072
{'the_donald': 52, 'altright': 7, 'trumpgret]https': 1, 'videos': 46, 'theredpill': 7, 'watchpeopledie': 4, 'incels': 4, 'physical_removal': 2, 'nfl': 9, 'teenagers': 4, 'gaming': 28, 'circlebroke2': 8, 'smashbros': 7, 'chapotraphouse': 14, 'conspiracy': 107, 'frenworld': 1, 'worldnews': 34, 'asktrumpsupporters': 3, 'politics**': 1, 'uncensorednews': 8, 'leagueoflegends': 23, 'battlefield': 6, 'wikileaks': 3, 'unexplainedphotos': 1, 'makenewfriendshere': 1, 'games': 31, 'amitheasshole': 13, 'kotakuinaction': 64, 'cringeanarchy': 16, 'rickandmorty': 14, 'iamatotalpieceofshit': 5, 'joerogan': 2, 'pizzagate': 1, 'shoplifting': 4, 'television': 22, 'pics': 67, 'pcgaming': 16, 'me_irl': 3, 'unpopularopinion': 7, 'deepfakes': 1, 'greatawakening': 8, 'hearthstone': 8, 'rainbow6': 3, 'fitgirls': 2, 'freefolk': 4, 'livestreamfail': 16, 'marchagainsttrump': 2, 'socialism': 10, 'memes': 1, 'goldtesting': 1, 'morbidquestions': 7, 'iama': 42, 'stopgaming': 1, 'coachella': 1, 'ic